# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# TO-DO: Create a Keyspace 
query = """CREATE KEYSPACE IF NOT EXISTS sparkify WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
session.execute(query)

#### Set Keyspace

In [7]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace("sparkify")

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
# TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
# sessionId = 338, and itemInSession = 4
# Crate the needed for query 1
query = "CREATE TABLE IF NOT EXISTS song_playlist_item_session"
query += '''( session_id int,
              item_in_session int,
              artist text, 
              length float,  
              song text,  
            PRIMARY KEY(session_id,item_in_session))'''
session.execute(query)

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_item_session(session_id, item_in_session, artist, length, song)"
        query = query + "VALUES(%s, %s, %s, %s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), float(line[5]), str(line[9])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """SELECT artist, length, song
           FROM song_playlist_item_session
           WHERE session_id = 338 AND item_in_session = 4"""
rows = session.execute(query)
df_rows = []
for row in rows:
    df_rows.append((row[0], row[1], row[2]))
    
# Print out the first 5 result of query in DataFrame
pd.DataFrame(df_row, columns=['artist','song','length']).head()

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [23]:
# TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
# for userid = 10, sessionid = 182 
# Created needed table for query 2
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query += '''( user_id int,
              session_id int, 
              item_in_session int,
              artist text,
              first_name text,
              last_name text,
              song text,
           PRIMARY KEY((user_id, session_id), item_in_session))'''

session.execute(query)

In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session(user_id, session_id, item_in_session, artist, first_name,last_name, song)"
        query = query + "VALUES(%s, %s, %s, %s,%s, %s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), str(line[0]), str(line[1]), str(line[4]), str(line[9])))

In [37]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """ SELECT artist, song, first_name, last_name
           FROM song_playlist_session
           WHERE user_id = 10 AND session_id = 182
           order by item_in_session DESC """
rows = session.execute(query)
df_rows = []
for row in rows:
    df_rows.append((row[0], row[1], row[2], row[3]))
    
# Print out the result of query in DataFrame
pd.DataFrame(df_row)

,0,1,2
0,The Academy Is...,Paper Chase (Album Version),209.762817
1,Cold,Stupid Girl,189.831375
2,Wonderwall,Just More (Album Version),281.024841
3,Evanescence,My Immortal,262.791382
4,Jason Mraz & Colbie Caillat,Lucky (Album Version),189.622406
5,Pixies,Break My Body,113.788933
6,Matt Redman,You Never Let Go,289.357910
7,Digitalism,Moonlight,171.963623
8,Jim White,Still Waters,396.224854
9,Sting,Windmills Of Your Mind,257.122803


In [14]:
# TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# Created needed table for query 3
query = "CREATE TABLE IF NOT EXISTS song_playlist_library"
query += '''( song text,
              user_id int,
              first_name text,
              last_name text,
           PRIMARY KEY(song, user_id))'''

session.execute(query)

In [15]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_library(song, user_id, first_name, last_name)"
        query = query + "VALUES(%s,%s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (str(line[9]), int(line[10]), str(line[1]), str(line[4])))

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """ SELECT first_name, last_name
           FROM song_playlist_library
           WHERE song = 'All Hands Against His Own' """

rows = session.execute(query)
df_rows = []
for row in rows:
    df_rows.append(row)
# Print out the result of query in DataFrame
pd.DataFrame(df_rows)

,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [17]:
## TO-DO: Drop the table before closing out the sessions
table_names = ['app_history', 'user_table', 'song_playlist_library']
for i in table_names:
    session.execute("""DROP TABLE IF EXISTS {}""".format(i))

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()